In [ ]:
import numpy as np
import json

In [ ]:
f = np.asarray([0.1,-0.3,-0.4,0,0.2])
#f = np.asarray([[0.1,-0.3,-0.4,0,0.2],[-0.4,0,0.2,0.5,0.6]])
#f = np.asarray([0.1,-0.3,-0.4,0,0.2,0.5,0.6])
topk = 4
filename = 'data.json'

In [ ]:
def ReLU(X):
   return np.maximum(0,X)

In [ ]:
def CReLU(f):
    f_pos = f
    f_neg = -f
    final_f = np.concatenate((f_pos, f_neg))
    return ReLU(final_f)

In [ ]:
def getSurrogateText(F,topK):
    surrText = ""
    count = topK
    for ele in F:
        instanceText = "RO"+str(ele)
        instanceText += (" "+instanceText)*(count-1)
        if len(surrText) == 0:
            surrText = instanceText
        else:
            surrText += (" "+instanceText)
        count -= 1
    return surrText

In [ ]:
def F2Text(f,topK):
    rf = CReLU(f)
    perVec = np.argsort(rf)[::-1]+1
    invPer = np.argsort(perVec)+1
    after_topk = invPer[invPer <= topK]
    return getSurrogateText(after_topk,topK)

In [ ]:
def getElasticData(fv,topK):
    nText = fv.shape[0]
    for i in range(nText):
        #random.shuffle(li)
        surrText = F2Text(fv[i],topK)
        data1 = {"index":{"_id":i+1}}

        data2 = {
          "text": surrText,
          "image": "ImagePath_"+str(i+1),
        }

        with open(filename,'a') as f: 
            json.dump(data1, f)
            f.write('\n')
            json.dump(data2, f)
            f.write('\n')

In [ ]:
def genFakeData(nSamples,topK):
    # It save generate data in file.
    # fire in termina --> file_name: data.json, Index: cbir, type: surrtext. (bulk load in elasticsearch)
    # curl -H 'Content-Type: application/x-ndjson' -XPOST 'localhost:9200/cbir/surrtext/_bulk?pretty' --data-binary @data.json

    # check in kibana
    # GET /_cat/indices
    # GET /cbir/surrtext/5

    # query:
    #GET /cbir/surrtext/_search
    #{
    #  "query":{
    #    "match":{
    #      "text":"RO1 RO1 RO1 RO1 RO1 RO5 RO5 RO5 RO5 RO3 RO3 RO3 RO4 RO4 RO2"
    #    }
    #  }
    #}
    
    nText = nSamples
    li = np.asarray([1,2,3,4,5])
    for i in range(nText):
        np.random.shuffle(li)
        surrText = F2Text(li,topK)
        data1 = {"index":{"_id":i+1}}

        data2 = {
          "text": surrText,
          "image": "ImagePath_"+str(i+1),
        }

        with open(filename,'a') as f: 
            json.dump(data1, f)
            f.write('\n')
            json.dump(data2, f)
            f.write('\n')

In [ ]:
surrText = F2Text(f,topk)
print(surrText)

RO4 RO4 RO4 RO4 RO3 RO3 RO3 RO2 RO2 RO1


In [ ]:
#getElasticData(f,topk)

In [ ]:
#genFakeData(15,topk)